#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.callback import callback, callback_function
from arena_api.system import system


### Callbacks: On Node Change
>    This example demonstrates configuring a callback to be invoked when a node
    is invalidated. A node is invalidated when its value changes or can be
    invalidated manually. In this example, a callback is registered on
    PayloadSize. The example shows two ways to invoke a callback: first by
    changing the value of a dependent node (Height) and then by invalidating
    PayloadSize manually. Whenever the callback is triggered, the callback
    function prints the updated value of the invalidated node.

In [ ]:
TAB1 = "  "
TAB2 = "    "
height_one = 256
height_two = 512

In [ ]:
"""
Waits for the user to connect a device before 
raising an exception
"""
tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                    '.' * sec_count, end='\r')
        tries += 1
    else:
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

device = devices[0]

##### Must have the decorator on the callback function
> node.on_update requires node as its first parameter<br>
This function is triggered when the callback event is triggered

In [ ]:
@callback_function.node.on_update
def print_node(node, *args, **kwargs):

    print(f'{TAB1}{TAB2}Message from callback')
    print(f'{TAB2}{TAB2}{node.name} : {str(node.value)}')

##### Store initial values
> These initial values are restored to the device after the example is completed

In [ ]:
nodemap = device.nodemap
nodes = nodemap.get_node(["Height", "PayloadSize"])

if (nodes["PayloadSize"].is_readable is False):
    raise Exception("PaytloadSize not readable")

height_initial = nodes["Height"].value

##### Demonstrates callbacks invoked on node changes
> - registers callback on node PayloadSize
> - changes Height twice to invalidate PayloadSize, invoking callback
> - invalidates PayloadSize manually
> - deregisters callback
"""

##### Register the callback on event node
> - Register PayloadSize for callbacks
> > - Callbacks are registered with a node and a function. This example demonstrates callbacks being invoked when the node is invalidated. This could be when the node value changes, either manually or by the device, or when the node is invalidated manually.

In [ ]:
print(f"{TAB1}Register Callback on PayloadSize")
handle = callback.register(nodes["PayloadSize"], print_node)

##### Invoke callbacks
> - Modify Height to invoke callback on PayloadSize
> > - The value of PayloadSize depends on a number of other nodes. This includes Height. Therefore, changing the value of Height changes the value of and invalidates PayloadSize, which then invokes the callback.
> - Manually invalidate PayloadSize for callback
> > - Apart from changing the value of a node, nodes can be invalidated manually by calling InvalidateNode. This also invokes the callback.

In [ ]:
print(f"{TAB2}Change Height Once")
nodes["Height"].value = height_one

print(f"{TAB2}Change Height Twice")
nodes["Height"].value = height_two

print(f"{TAB2}Invalidate PayloadSize")
nodes["PayloadSize"].invalidate_node()

##### Deregister callback
> - Failing to deregister a callback results in a memory leak. Once a callback has been registered, it will no longer be invoked when a node is invalidated

In [ ]:
print(f'{TAB2}Deregister Callback')
callback.deregister(handle)

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
nodes["Height"].value = height_initial

system.destroy_device(device)